In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [2]:
!rm -rf sample_data
!rm -rf mmdb_400
!rm -rf mmdb_400_96
!rm -rf mmdb_400_96_bw
# !rm -rf segmented
# !rm -rf segmented_224
# !rm -rf random.png random_data.png

In [ ]:
# !unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/combined_segmented.zip'
# !unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/png.zip'
# !unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/bw_segmented.zip'
# !unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/tempdb.zip'
# !unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/mmdb_400.zip'
# !unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/mmdb_400_96.zip'
!unzip '/content/drive/MyDrive/Colab Notebooks/Final_Year_Project/mmdb_400_96.zip'

In [4]:
# import os

# root_dir = os.listdir()
# # os.rename('combined_segmented', 'segmented')
# # os.rename('png', 'segmented')
# # os.rename('bw_segmented', 'segmented')
# os.rename('tempdb', 'segmented')
# # os.rename('testdb', 'segmented')

In [15]:
import numpy as np
import os
# from imageio import imread
# from keras.utils import to_categorical
# from skimage.transform import resize
import glob
import math
import cv2
import random
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import keras
from keras.applications.resnet_v2 import preprocess_input
from keras.applications.resnet_v2 import ResNet152V2
from keras import Model
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras import layers, models
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping,  ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline

In [ ]:
# Loading the data and generating labels

IMG_SIZE = 96
image_height, image_width = IMG_SIZE, IMG_SIZE
no_of_color_channels = 3
root_dir = os.getcwd()
db_dir = f"{root_dir}/mmdb_400_96"
no_of_classes = len(os.listdir(db_dir))
print(root_dir)

total_no_of_images = len(list(glob.glob(f"{db_dir}/[0-9]*/*.*", recursive=True)))
print(total_no_of_images)

count = 0
# test_size = 2 # from each class
test_size = int((total_no_of_images/56)*10/100)  # from each class
train_size = total_no_of_images - no_of_classes * test_size
data = np.empty((train_size, image_height, image_width, no_of_color_channels))
labels = np.empty(train_size, dtype=int)
# data = []
# labels = []

test_count = 0
test_data = np.empty((no_of_classes*test_size, image_height, image_width, no_of_color_channels))
test_labels = np.empty(no_of_classes*test_size, dtype=int)
# test_data = []
# test_labels = []

print(train_size)
print(no_of_classes*test_size)

for i in range(no_of_classes):
    class_dir = f"{db_dir}/{i}"
    class_images = glob.glob(class_dir+'/*.*')
    np.random.shuffle(class_images)
    
    # class_images = [cv2.resize(cv2.imread(image_path), (32, 32)) for image_path in class_images]
    # test_data = [*test_data, *class_images[:test_size]]
    # test_labels = [*test_labels, *[i]*len(class_images[:test_size])]
    # data = [*data, *class_images[test_size:]]
    # labels = [*labels, *[i]*len(class_images[test_size:])]
    
    size = len(class_images)
    print(i, end=" ") if i == 0 or i % 10 != 0 else print(i)
    for j in range(size):
        image_path = class_images[j]
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        if j < size-test_size:
          data[count] = image
          labels[count] = i
          count += 1
        else:
          test_data[test_count] = image
          test_labels[test_count] = i
          test_count += 1
print('')

In [ ]:
data = np.array(data)
test_data = np.array(test_data)
labels = np.array(labels)
test_labels = np.array(test_labels)
print(data.shape, test_data.shape)
print(labels.shape, test_labels.shape)

In [18]:
def tempDisplay(dict_):
    """
    - `Input`: dictionary
        - ``key``: name for the image to be displayed
        - ``value``: image to be displayed
    - `Output`: a matplotlib plot with the all the images in the dictionary in one image
    """
    rows = int(math.sqrt(len(dict_)))
    cols = math.ceil(len(dict_)/rows)
    fig = plt.figure(figsize=(10, 5))
    i = 1
    for key in dict_:
        fig.add_subplot(rows, cols, i)
        i = i + 1
        plt.imshow(dict_[key])
        plt.axis('off')
        plt.title(str(key))
    plt.show()

In [19]:
# Shuffle Data

data, labels = shuffle(data, labels)
# test_data, test_labels = shuffle(test_data, test_labels)

In [ ]:
r = random.randrange(train_size)
cv2.imwrite(root_dir+'/random_image.png', data[r])
# plt.imshow(data[r])
tempDisplay({"data[r]": np.asarray(data[r])})

In [ ]:
# Preprocess training and testing data

print(data.min(), data.max())
print(test_data.min(), test_data.max())

# for i in range(len(data)):
#   data[i] = preprocess_input(data[i])
# for i in range(len(test_data)):
#   test_data[i] = preprocess_input(test_data[i])

data = preprocess_input(data)
test_data = preprocess_input(test_data)

print(data.min(), data.max())
print(test_data.min(), test_data.max())

In [ ]:
plt.imshow(data[r])

In [23]:
# np.save(root_dir+'/data.npy', data)
# np.save(root_dir+'/labels.npy', labels)
# np.save(root_dir+'/test_data.npy', test_data)
# np.save(root_dir+'/test_labels.npy', test_labels)
# # data = np.load(root_dir+'/data.npy')
# # labels = np.load(root_dir+'/labels.npy')

In [ ]:
 print(data[random.randrange(total_no_of_images)].shape)

print(data.min(), data.max())
print(test_data.min(), test_data.max())
print(labels)
print(len(test_data), test_data[random.randrange(no_of_classes)].shape)
# print(test_labels)

img = np.array(cv2.imread(root_dir+'/random_image.png'))
x = preprocess_input(img)
cv2.imwrite(root_dir+'/p_random_image.png', x)
p_img = np.array(cv2.imread(root_dir+'/p_random_image.png'))
print(img.min(), img.max())
print(p_img.min(), p_img.max())

tempDisplay({"img": img, "p_img": p_img})

# fig = plt.figure(figsize=(5, 5))
# fig.add_subplot(1, 2, 1)
# plt.imshow(img)
# plt.axis('off')
# fig.add_subplot(1, 2, 2)
# plt.imshow(p_img)
# plt.axis('off')

In [ ]:
# Importing Resnet152V2

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = ResNet152V2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False
base_model.summary()

In [ ]:
# Modifying the model

avgpooling_layer_1 = GlobalAveragePooling2D()
dense_layer_1 = Dense(1024, activation='relu')
dense_layer_2 = Dense(1024, activation='relu')
dense_layer_3 = Dense(512, activation='relu')
prediction_layer = Dense(no_of_classes, activation='softmax')

character_model = models.Sequential([
    base_model,
    avgpooling_layer_1,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    prediction_layer,
])
character_model.summary()

In [27]:
# Compiling the model

# learning_rate = 0.0001
# epochs=200
# decay_rate = learning_rate / epochs
# momentum = 0.8
# custom_sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate)

# custom_adam = Adam(learning_rate=0.0000025)
custom_adam = Adam(learning_rate=0.00025)

character_model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer='adam',
    optimizer=custom_adam,
    metrics=['accuracy']
)

In [28]:
# Callbacks: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint('resnet_152v2_mmdb_400.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
# rlronp = ReduceLROnPlateau( monitor="val_accuracy", factor=0.005,patience=1, verbose=1)
rlronp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1,  min_lr=0.00001)

callbacks = [earlystop, checkpoint, rlronp]

In [ ]:
# Training the model
history = character_model.fit(
    x=data,
    y=labels,
    epochs=200,
    shuffle=True,
    verbose='auto',
    callbacks=callbacks,
    validation_split=0.1,
)

In [ ]:
print(history.history.keys())
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(37) #change accordingly of epochs that run 
plt.plot(epochs, loss_train, 'g', label='Training Loss')
plt.plot(epochs, loss_val, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
print(history.history.keys())
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(37) #change accordingly of epochs that run  
plt.plot(epochs, loss_train, 'g', label='Training Accuracy')
plt.plot(epochs, loss_val, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [33]:
character_model.save('resnet_152v2_mmdb_400.h5')

In [ ]:
score = character_model.evaluate(test_data, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predictions = character_model.predict(test_data)
print('Shape: {}'.format(predictions.shape))

In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sns

def plot_confusion_matrix(y_test,y_scores, classNames):
    # y_test=np.argmax(y_test, axis=1)
    # y_scores=np.argmax(y_scores, axis=1)
    classes = len(classNames)
    cm = confusion_matrix(y_test, y_scores)
    # print("**** Confusion Matrix ****")
    # print(cm)
    # print("**** Classification Report ****")
    # print(classification_report(y_test, y_scores, target_names=classNames))
    con = np.zeros((classes,classes))
    for x in range(classes):
        for y in range(classes):
            con[x,y] = cm[x,y]/np.sum(cm[x,:])

    plt.figure(figsize=(100,100))
    sns.set(font_scale=2.0) # for label size
    df = sns.heatmap(con, annot=True,fmt='.2', cmap='Blues',xticklabels= classNames , yticklabels= classNames)
    df.figure.savefig("image2.png")

predicted_classes = np.argmax(predictions, axis=1)
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_labels, predicted_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_labels, predicted_classes, average="macro")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_labels, predicted_classes, average="macro")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_labels, predicted_classes, average="macro")
print('F1 score: %f' % f1)
# confusion matrix
plot_confusion_matrix(test_labels, predicted_classes, [str(i) for i in range(56)])
 
# # kappa
# kappa = cohen_kappa_score(test_labels, predicted_classes)
# print('Cohens kappa: %f' % kappa)
# # ROC AUC
# auc = roc_auc_score(test_labels, predicted_classes, multi_class="ovr")
# print('ROC AUC: %f' % auc)
# matrix = multilabel_confusion_matrix(test_labels, predicted_classes)
# # for i in range(len(matrix)):
# #   for j in range(len(matrix[0])):
# #     print(matrix[i][j], end=" ")
# #   print("")
# print(matrix)

In [ ]:
print(classification_report(test_labels, predicted_classes))

In [ ]:
for i in range(no_of_classes*test_size):
  output_neuron = np.argmax(predictions[i])
  print('Most active neuron: {} ({:.2f}%)'.format(
      output_neuron,
      100 * predictions[i][output_neuron]
  ))

In [ ]:
top1_correct = 0
top1_incorrect = 0
top5_correct = 0
top5_incorrect = 0
for i in range(no_of_classes*test_size):
  s = set()
  sorted_predictions = np.sort(predictions[i])[::-1]
  for j in range(5):
    x = list(np.where(predictions[i]==sorted_predictions[j]))[0][0]
    s.add(x)
  # print(list(s))
  if test_labels[i] in s: top5_correct+=1
  else: top5_incorrect+=1
  if test_labels[i] == np.argmax(predictions[i]): top1_correct+=1
  else: top1_incorrect+=1

print('Top-1 Accuracy: ', (top1_correct*100)/(top1_correct+top1_incorrect))
print('Top-5 Accuracy: ', (top5_correct*100)/(top5_correct+top5_incorrect))